## Import libraries

In [131]:
import time
import pandas as pd
import re

In [132]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

## Scroll function

In [133]:
def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

## Locate ChromeDriver and open URL

In [260]:
driver = webdriver.Chrome('/Users/kimberlyngoh/Downloads/chromedriver')  
# Optional argument, if not specified will search path.

In [261]:
url = 'https://www.sephora.com/shop/bath-body'
#url = 'https://www.sephora.com/product/kp-bump-eraser-body-scrub-with-10-aha-P444989?icid2=products%20grid:p444989'
driver.get(url)

In [262]:
xpath = '/html/body/div[3]/div[5]/div/div/div/div[1]/div/div/nav/div[5]/a'
btn = driver.find_element_by_xpath(xpath)
btn.click()
#xpath2 = '/html/body/div[3]/div[5]/div/div/div/div[2]/div[1]/div/nav/a[1]'
#btn = driver.find_element_by_xpath(xpath2)
#btn.click()

## Create empty dataframe

In [91]:
df = pd.DataFrame(columns=['Label', 'URL'])
print(df)

Empty DataFrame
Columns: [Label, URL]
Index: []


## Web-scrape labels and URLs

In [92]:
tickers = ['bath-and-body-soap', 'body-moisturizers', 'sun-lotion', 'body-care','beauty-supplements-bath-body']

In [93]:
for ticker in tickers:
    url = 'https://www.sephora.com/shop/' + ticker + '?pageSize=300'
    driver.get(url)
    
    # xpath = '/html/body/div[3]/div[5]/div/div/div/div[2]/div[1]/div/nav/a[1]'
    #/html/body/div[3]/div[5]/div/div/div/div[2]/div[1]/main/div[2]/div/div[1]/div[1]/div[1]/a
    # btn = driver.find_element_by_xpath(xpath)
    # btn.click()
    #time.sleep(20)

    browser = scrollDown(driver, 10)
    time.sleep(10)

    browser = scrollDown(driver, 10)
    time.sleep(10)

    browser = scrollDown(driver, 10)
    time.sleep(10)

    browser = scrollDown(driver, 10)

    element = driver.find_elements_by_class_name('css-ix8km1')

    subpageURL = []
    for a in element:
        subURL = a.get_attribute('href')
        subpageURL.append(subURL)
        
        # transform into a data frame
    dic = {'Label': ticker, 'URL': subpageURL}
    df = df.append(pd.DataFrame(dic), ignore_index = True)

# add columns
df2 = pd.DataFrame(columns=['brand', 'name', 'price', 'skin_type', 'ingredients'])
df = pd.concat([df, df2], axis = 1)

## Save file (initial) / Read file (later)

In [143]:
# df.to_csv('body-care-url.csv', encoding = 'utf-8-sig', index = False)
df3 = pd.read_csv('body-care-1.csv')

## Test block

In [232]:
url = df3.URL[0]
driver.get(url)
time.sleep(5)

browser = scrollDown(driver, 1)
time.sleep(5)

# ingredients
xpath = '//*[@id="tab2"]'
btn = driver.find_element_by_xpath(xpath)
btn.click()

try:
    df3.ingredients[0] = driver.find_element_by_xpath('//*[@id="tabpanel2"]/div').text

except NoSuchElementException:
    df3.ingredients[0] = 'No Info'

In [292]:
df3.head(665)

,Label,URL,brand,name,price,skin_type,ingredients
0,bath-and-body-soap,https://www.sephora.com/product/kp-bump-eraser...,FIRST AID BEAUTY,KP Bump Eraser Body Scrub with 10% AHA,$28.00,"[Normal,, Dry,, Combination\n]",-Pumice Buffing Beads: Exfoliate particles to ...
1,bath-and-body-soap,https://www.sephora.com/product/sol-de-janeiro...,SOL DE JANEIRO,Bum Bum Body Scrub,$42.00,"[Normal,, Dry,, Combination,, Oily\n]",-Crushed Cupuaçu Seeds: Gently buff and smooth...
2,bath-and-body-soap,https://www.sephora.com/product/brazilian-4-pl...,SOL DE JANEIRO,Brazilian 4 Play Moisturizing Shower Cream-Gel,$25.00,"[Dry,, Normal,, Combination,, Oily,]",
3,bath-and-body-soap,https://www.sephora.com/product/sol-de-janeiro...,SOL DE JANEIRO,Brazilian Touch Hand Sanitizer Spray,$10.00,[],"Alcohol 80%, Aqua (water), glycerine, hydrogen..."
4,bath-and-body-soap,https://www.sephora.com/product/necessaire-the...,NÉCESSAIRE,The Body Wash,$25.00,"[Normal,, Dry,, Combination,, Oily\n]","-Niacinamide (Vitamin B3): Cleanses, nourishes..."
...,...,...,...,...,...,...,...
660,beauty-supplements-bath-body,https://www.sephora.com/product/uber-energy-su...,HUM NUTRITION,Uber Energy Adrenal Fatigue and Adaptogen Supp...,$25.00,[],-Amino Acid: Helps balance stress hormones.*\n...
661,beauty-supplements-bath-body,https://www.sephora.com/product/beauty-collage...,VITAL PROTEINS,Beauty Collagen - Lavender Lemon,$32.00,[],-Hyaluronic Acid: Moisturizes.\n-Bacillus Coag...
662,beauty-supplements-bath-body,https://www.sephora.com/product/skinny-bird-su...,HUM NUTRITION,Skinny Bird™ Supplements,$40.00,[],-5-HTP: Reduces stress eating.\n-Green Tea Ext...
663,beauty-supplements-bath-body,https://www.sephora.com/product/beauty-collage...,VITAL PROTEINS,Beauty Collagen - Tropical Hibiscus,$32.00,[],-Hyaluronic Acid: Provides moisture.\n-Bacillu...


## Web-scrape brand, name, price, skin-type, ingredients

In [288]:
for i in range(len(df3)+1):
    url = df3.URL[i]
    driver.get(url)
    time.sleep(5)
    
    try:    
    # brand, name, price
        df3.brand[i] = driver.find_element_by_class_name('css-es084o').text
        df3.name[i] = driver.find_element_by_css_selector('span.css-0').text
        df3.price[i] = driver.find_element_by_class_name('css-slwsq8').text
    
    except NoSuchElementException:
        df3.brand[i] = 'No Info'
        df3.name[i] = 'No Info'
        df3.price[i] = 'No Info'

    browser = scrollDown(driver, 1)
    time.sleep(5)
    
    # skin_type
    try: 
        detail = driver.find_element_by_class_name('css-pz80c5').text
        pattern = r"(Normal\W|Dry\W|Combination\W|Oily\W)"
        df3.skin_type[i] = re.findall(pattern,detail)
    
    except NoSuchElementException:
        df3.skin_type[i] = 'No Info'
    
    # ingredients
    xpath = '//*[@id="tab2"]'
    btn = driver.find_element_by_xpath(xpath)
    btn.click()

    try:
        df3.ingredients[i] = driver.find_element_by_xpath('//*[@id="tabpanel2"]/div').text

    except NoSuchElementException:
        df3.ingredients[i] = 'No Info'

    #print(i)    # just for verbose

290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539


KeyError: 666

## Save file

In [289]:
df3.to_csv('body-care-1.csv', encoding = 'utf-8-sig', index = False)